### Creacion de nuevas columnas en un dataFrame

#### Creacion de una columna con un valor constante

Si bien es muy poco usual hacer este tipo de cosas, puede ser util saber hacerlo para generar una columna con un determinado valor de default por poner un ejemplo tipico.

Como vemos en el ejemplo que sigue al setear el indice usamos el atributo inplace=True, esto es para que en la misma linea haga el seteo sin necesidad de asignar el resultado del mismo a la misma variable data, es un seteo con asignacion inline como se dice.

Lo del drop(['high', 'low'], axis=1) es simplemente borrar esas dos colunas para que me entre mejor en la pagina la tabla y limpiar un poco de datos que no necesito ver.


In [ ]:
import pandas as pd
data = pd.read_excel('AAPL.xlsx', sheet_name='Hoja1')
data.set_index('timestamp', inplace=True)
data['Precio_COME']=3
data.drop(['high', 'low'], axis=1).head()

Drop es borrar obviamente, el listado de columnas va entre corchetes porque es una lista, y luego uso axis=1 indicando que me refiero a columnas, ya que si quiero borrar filas uso axis=0 que es el valor default de la variable axis.

Tengamos en cuenta siempre lo siguiente, cuando escribimos en la Jupyter notebook:


In [ ]:
data.drop(['high', 'low'], axis=1).head()

Directamente me "da salida" y me imprime la tabla como vimos, pero si lo hacemos en un IDE sin poner print no nos devuelve nada... pero hay mas, al hacerlo sin haberle asignado esa salida a "data" si ahora le volvemos a pedir que imprima data van a aparecer las columnas borradas nuevamente, porque lo que hicimos fue solamente ver la salida de la linea pero no se lo asignamos a la variable, para ello deberiamos haber escrito:


In [ ]:
data = data.drop(['high', 'low'], axis=1).head()
data

#### Creacion de una columna a partir de otras columnas

Tambien se puede crear una columna nueva partiendo de los valores de otra u otras columnas. Veamos unos ejemplos.

Creamos una columna que muestre el precio de apertura, cierre, maximo y minimo
